#### 1. 定义相关函数，获取数据库基本信息

In [1]:
import requests
import json

# 定义请求的URL
url = "http://cloud.gstore.cn/api"

# 定义AccessKeyID和AccessSecret
access_key_id = "00b10697151c4791bb55beee95d136bd"
access_secret = "65194C5F25B1CE0C90011D34ACBED4EC"

# 定义请求头
headers = {
    "Content-Type": "application/json"
}

# 根据给定的 sparq 语句获取对应的输出
def sql(sparql):
    params = {
        "action": "queryDB",
        'dbName': '金融数据库',
        'accesskeyid': access_key_id,
        'access_secret': access_secret,
        'sparql': sparql
    }

    json_params = json.dumps(params)
    response = requests.post(url, headers=headers, data=json_params).json()
    return response

# 判断某一公司实体是否存在于数据库中
def entity_exist(company):
    ask = f'''
    ASK
    WHERE {{
      <file:///F:/d2r-server-0.7/holder8.nt#holder_copy/{company}> ?p ?o .
    }}
    '''
    response = sql(ask)
    return response['data']['results']['bindings'][0]['_askResult']['value'] == 'true'

In [2]:
# 获取数据库基本信息
params = {
    "action": "monitorDB",
    'dbName': '金融数据库',
    'accesskeyid': access_key_id,
    'access_secret': access_secret
}

json_params = json.dumps(params)
response = requests.post(url, headers=headers, data=json_params).json()
entitynum = response['entityNum'] 
triplenum = response['tripleNum']
print(f'实体数量{entitynum}， 三元组数量{triplenum}')

实体数量16652， 三元组数量21440


In [3]:
# 获取实体构成的列表
ask_1 = '''SELECT DISTINCT ?entity
WHERE {
  ?entity ?p ?o .
}
'''
response = sql(ask_1)
entity_list = [binding['entity']['value'][49:] for binding in response['data']['results']['bindings'][2:]]
for i in range(3):
    print(entity_list[i]) # 输出列表中前 3 个公司

安邦财产保险股份有限公司
招商银行股份有限公司
和谐健康保险股份有限公司


#### 2. 任务1：查询两个公司之间的关联路径（n-hop）, n>1

In [45]:
from graphviz import Digraph
import matplotlib.colors as mcolors

# 两跳查询
def query_2_hop(entity_1,entity_2):
    if not entity_exist(entity_1):
        print(f'{entity_1}不在数据库中')
        return []
    if not entity_exist(entity_2):
        print(f'{entity_2}不在数据库中')
        return []

    ask = f'''
    SELECT DISTINCT ?start_entity ?mid_entity ?end_entity
    WHERE {{
    {{
        ?start_entity ?p1 ?mid_entity .
        ?mid_entity ?p2 ?end_entity .
        FILTER (?start_entity = <file:///F:/d2r-server-0.7/holder8.nt#holder_copy/{entity_1}> &&
                ?end_entity = <file:///F:/d2r-server-0.7/holder8.nt#holder_copy/{entity_2}>)
    }} UNION {{
        ?start_entity ?p1 ?mid_entity .
        ?mid_entity ?p2 ?end_entity .
        FILTER (?start_entity = <file:///F:/d2r-server-0.7/holder8.nt#holder_copy/{entity_2}> &&
                ?end_entity = <file:///F:/d2r-server-0.7/holder8.nt#holder_copy/{entity_1}>)
    }}
    }}
    '''
    response = sql(ask)
    res = [(binding['start_entity']['value'][49:],binding['mid_entity']['value'][49:],binding['end_entity']['value'][49:]) for binding in response['data']['results']['bindings']]
    return res

# n跳查询，n > 1
def query_n_hop(entity_1, entity_2,n = 3):
    if not entity_exist(entity_1):
        print(f'{entity_1}不在数据库中')
        return []
    if not entity_exist(entity_2):
        print(f'{entity_2}不在数据库中')
        return []

    # 构造查询语句
    query_con = ''
   # 构造每一跳的模式
    for i in range(1, n+1):
        if i == 1:
            query_con += f"  ?start_entity ?p1 ?mid1_entity .\n"
        elif i == n:
            query_con += f"  ?mid{i-1}_entity ?p{i} ?end_entity .\n"
        else:
            query_con += f"  ?mid{i-1}_entity ?p{i} ?mid{i}_entity .\n"
    
    mid_entity_list = [f'mid{i}_entity' for i in range(1, n)] # 中间变量列表
    query = f'''
    SELECT DISTINCT {' '.join(['?start_entity'] + ['?' + e for e in mid_entity_list] + ['?end_entity'])}
    WHERE {{
        {{  
            {query_con}
            FILTER (?start_entity = <file:///F:/d2r-server-0.7/holder8.nt#holder_copy/{entity_1}> &&
                    ?end_entity = <file:///F:/d2r-server-0.7/holder8.nt#holder_copy/{entity_2}>)
        }}
        UNION
        {{  
            {query_con}
            FILTER (?start_entity = <file:///F:/d2r-server-0.7/holder8.nt#holder_copy/{entity_2}> &&
                    ?end_entity = <file:///F:/d2r-server-0.7/holder8.nt#holder_copy/{entity_1}>)
        }}
    }}
    '''

    response = sql(query)
    res = []
    for binding in response['data']['results']['bindings']:
        item = (binding['start_entity']['value'][49:],) + tuple(binding[e]['value'][49:] for e in mid_entity_list[:n]) + (binding['end_entity']['value'][49:],)
        res.append(tuple(item))
    return res

# 生成颜色渐变列表
def generate_color_gradient(num_colors):
    colors = list(mcolors.CSS4_COLORS.values())
    bright_colors = [color for color in colors if "black" not in color.lower() and "dark" not in color.lower()]
    return bright_colors[:num_colors]

# 图形化表示
def format_as_graph(results):
    max_hops = max(len(result) for result in results)
    colors = generate_color_gradient(max_hops)

    dot = Digraph(comment='N-hop Query Results')
    
    drawn_nodes = set()
    for path_index, result in enumerate(results):
        path_label = f" {path_index + 1}"
        for i in range(len(result) - 1):
            # 根据层级选择颜色
            start_node_color = colors[i % len(colors)]
            end_node_color = colors[(i + 1) % len(colors)]

            # 添加起始节点
            if result[i] not in drawn_nodes:
                dot.node(result[i], style='filled', color=start_node_color, fontsize='12')
                drawn_nodes.add(result[i])
            
            # 添加结束节点
            if result[i + 1] not in drawn_nodes:
                dot.node(result[i + 1], style='filled', color=end_node_color, fontsize='12')
                drawn_nodes.add(result[i + 1])
            
            # 添加带有路径序号的边
            dot.edge(result[i], result[i + 1], label=path_label)
    
    return dot

In [5]:
res_1 = query_2_hop('招商局轮船股份有限公司', '招商银行股份有限公司')
res_2 = query_n_hop('招商局轮船股份有限公司', '招商银行股份有限公司',n=3)
for i in range(len(res_1)):
    print(res_1[i])
print('****')
for i in range(len(res_2)):
    print(res_2[i])

('招商局轮船股份有限公司', '深圳市招融投资控股有限公司', '招商银行股份有限公司')
('招商局轮船股份有限公司', '深圳市晏清投资发展有限公司', '招商银行股份有限公司')
****
('招商局轮船股份有限公司', '深圳市招融投资控股有限公司', '深圳市楚源投资发展有限公司', '招商银行股份有限公司')
('招商局轮船股份有限公司', '深圳市招融投资控股有限公司', '深圳市晏清投资发展有限公司', '招商银行股份有限公司')
('招商局轮船股份有限公司', '深圳市晏清投资发展有限公司', '深圳市楚源投资发展有限公司', '招商银行股份有限公司')


In [38]:
dot = format_as_graph(res_2)
dot.render('n_hop_query_results', format='png', cleanup=True)

'n_hop_query_results.png'

#### 3. 任务2:实现多层股权的穿透式查询，可以根据指定层数获得对应层级的股东

In [69]:
# 单层持股查询
def query_1_hop(entity):
  ask = f'''SELECT DISTINCT ?company
  WHERE {{
    <file:///F:/d2r-server-0.7/holder8.nt#holder_copy/{entity}> ?p ?company .
  }}
  '''
  res = [binding['company']['value'][49:] for binding in sql(ask)['data']['results']['bindings']]
  return res

# 多层持股查询
def query_multi_holder(entity,n = 3):
  res = [{entity: None}]
  visited = set()  # 初始化空的已访问公司集合
  visited.add(entity)
  current_list = [entity]  # 初始化待访问公司列表

  for _ in range(n):
      current_dict = {}  # 使用字典记录每层的公司及其上一层公司
      for query_entity in current_list:
          query_res = query_1_hop(query_entity)
          for result_entity in query_res:
              if result_entity not in visited:
                  current_dict[result_entity] = query_entity
                  visited.add(result_entity)
      current_list = list(current_dict.keys())
      res.append(current_dict)
  return res

# 图形化表示
def draw_tree(res):
    num_levels = len(res)
    colors = generate_color_gradient(num_levels)

    dot = Digraph(comment='Multi-Level Holder Query Results')
    dot.attr(size='20,30')  # 调整图形大小
    dot.attr(rankdir='TB')  # 树形图从上到下排列
    dot.attr('node', shape='box', style='filled', color='lightgrey', fontsize='12')  # 调整节点外观
    dot.attr('edge', fontsize='10')  # 调整边的字体大小

    drawn_nodes = set()
    
    for level, companies in enumerate(res):
        color = colors[level % len(colors)]  # 根据层级选择颜色
        for company, parent in companies.items():
            if company not in drawn_nodes:
                dot.node(company, style='filled', color=color, fontsize='12')
                drawn_nodes.add(company)
            if parent is not None and parent not in drawn_nodes:
                dot.node(parent, style='filled', color=color, fontsize='12')
                drawn_nodes.add(parent)
            if parent is not None:
                dot.edge(parent, company)

    return dot

In [71]:
entity = '招商局轮船股份有限公司'
res = query_multi_holder(entity, n = 3)

for level, companies in enumerate(res):
    print(f"层级 {level}:")
    for company, parent in companies.items():
        print(f"{parent} --> {company}")

层级 0:
None --> 招商局轮船股份有限公司
层级 1:
招商局轮船股份有限公司 --> 深圳市招融投资控股有限公司
招商局轮船股份有限公司 --> 深圳市晏清投资发展有限公司
招商局轮船股份有限公司 --> 兴业证券股份有限公司
招商局轮船股份有限公司 --> 招商局资本投资有限责任公司
层级 2:
深圳市招融投资控股有限公司 --> 招商银行股份有限公司
深圳市招融投资控股有限公司 --> 招商证券股份有限公司
深圳市招融投资控股有限公司 --> 深圳市楚源投资发展有限公司
兴业证券股份有限公司 --> 南方基金管理有限公司
招商局资本投资有限责任公司 --> 招商局资本控股有限责任公司
招商局资本投资有限责任公司 --> 招商局资本管理有限责任公司
层级 3:
招商银行股份有限公司 --> 招商基金管理有限公司
招商证券股份有限公司 --> 东北证券股份有限公司
招商证券股份有限公司 --> 中国南方航空股份有限公司
招商证券股份有限公司 --> 大连冷冻机股份有限公司
招商证券股份有限公司 --> 招商致远资本投资有限公司
招商证券股份有限公司 --> 博时基金管理有限公司
深圳市楚源投资发展有限公司 --> 深圳市集盛投资发展有限公司
南方基金管理有限公司 --> 南方资本管理有限公司
招商局资本控股有限责任公司 --> 国新国同（浙江）投资基金合伙企业（有限合伙）
招商局资本控股有限责任公司 --> 赣州招商致远壹号股权投资合伙企业（有限合伙）
招商局资本控股有限责任公司 --> 中新建招商股权投资有限公司
招商局资本管理有限责任公司 --> 深圳市招商致远股权投资基金管理有限公司


In [72]:
dot = draw_tree(res)
dot.render('multi_holder_tree', format='png', cleanup=True)

'multi_holder_tree.png'

#### 4.任务3: 实现环形持股查询，判断两家公司是否存在环形持股现象

In [73]:
# 查找存在的环路
def check_circular_holdings(entity_A, entity_C, n = 10):
    # 查询从 A 出发的多层股权关系
    result_A = query_multi_holder(entity_A, n = n)
    merged_result_A = {k: v for level_dict in result_A for k, v in level_dict.items()}
    # 查询从 C 出发的多层股权关系
    result_C = query_multi_holder(entity_C, n = n)
    merged_result_C = {k: v for level_dict in result_C for k, v in level_dict.items()}
    
    # 记录从 A 到 C 的路径
    path_A_to_C = []
    # 记录从 C 到 A 的路径
    path_C_to_A = []
    
    if entity_A in merged_result_C:
        current_entity = entity_A
        while current_entity is not None:
            path_C_to_A.insert(0, current_entity)
            current_entity = merged_result_C[current_entity]
    
    if entity_C in merged_result_A:
        current_entity = entity_C
        while current_entity is not None:
            path_A_to_C.insert(0, current_entity)
            current_entity = merged_result_A[current_entity]
    
    return  path_A_to_C,path_C_to_A

# 图形化表示
def draw_circular_path(path_A_to_C, path_C_to_A):
    colors = ['#F8B195', '#F67280', '#C06C84', '#6C5B7B', '#355C7D', '#BDC3C7', '#FFD700', '#FF8C00', '#FFB6C1', '#77DD77']
    dot = Digraph(comment='Circular Holdings Path')
    
    # 绘制从 A 到 C 的路径
    for i in range(len(path_A_to_C) - 1):
        start_node_color = colors[i % len(colors)]
        end_node_color = colors[(i + 1) % len(colors)]

        dot.node(path_A_to_C[i], style='filled', color=start_node_color, fontsize='12')
        dot.node(path_A_to_C[i + 1], style='filled', color=end_node_color, fontsize='12')
        dot.edge(path_A_to_C[i], path_A_to_C[i + 1], label='1')

    # 绘制从 C 到 A 的路径
    for i in range(len(path_C_to_A) - 1):
        start_node_color = colors[i % len(colors)]
        end_node_color = colors[(i + 1) % len(colors)]

        dot.node(path_C_to_A[i], style='filled', color=start_node_color, fontsize='12')
        dot.node(path_C_to_A[i + 1], style='filled', color=end_node_color, fontsize='12')
        dot.edge(path_C_to_A[i], path_C_to_A[i + 1], label='2')

    return dot

In [74]:
# 示例用法
entity_A = '招商局轮船股份有限公司'
entity_C = '招商银行股份有限公司'
path_A_to_C,path_C_to_A = check_circular_holdings(entity_A, entity_C)
if path_A_to_C and path_C_to_A:
    print(f"存在环形持股路径：")
    print(f"从 {entity_A} 到 {entity_C} 的路径：{' -> '.join(path_A_to_C)}")
    print(f"从 {entity_C} 到 {entity_A} 的路径：{' -> '.join(path_C_to_A)}")
elif path_A_to_C:
    print(f"只存在从 {entity_A} 到 {entity_C} 的路径：{' -> '.join(path_A_to_C)}")
elif path_C_to_A:
    print(f"只存在从 {entity_C} 到 {entity_A} 的路径：{' -> '.join(path_C_to_A)}")
else:
    print(f'{entity_A}与{entity_C}并不连通')

只存在从 招商局轮船股份有限公司 到 招商银行股份有限公司 的路径：招商局轮船股份有限公司 -> 深圳市招融投资控股有限公司 -> 招商银行股份有限公司


In [75]:
dot = draw_circular_path(path_A_to_C, path_C_to_A)
dot.render('circular_holdings_path_1', format='png', cleanup=True)

'circular_holdings_path_1.png'

In [76]:
# 示例用法
entity_A = '招商证券股份有限公司'
entity_C = '兖州煤业股份有限公司'
path_A_to_C,path_C_to_A = check_circular_holdings(entity_A, entity_C)
if path_A_to_C and path_C_to_A:
    print(f"存在环形持股路径：")
    print(f"从 {entity_A} 到 {entity_C} 的路径：{' -> '.join(path_A_to_C)}")
    print(f"从 {entity_C} 到 {entity_A} 的路径：{' -> '.join(path_C_to_A)}")
elif path_A_to_C:
    print(f"只存在从 {entity_A} 到 {entity_C} 的路径：{' -> '.join(path_A_to_C)}")
elif path_C_to_A:
    print(f"只存在从 {entity_C} 到 {entity_A} 的路径：{' -> '.join(path_C_to_A)}")
else:
    print(f'{entity_A}与{entity_C}并不连通')

存在环形持股路径：
从 招商证券股份有限公司 到 兖州煤业股份有限公司 的路径：招商证券股份有限公司 -> 大连冷冻机股份有限公司 -> 国泰君安证券股份有限公司 -> 上海证券交易所 -> 中国证券金融股份有限公司 -> 兖州煤业股份有限公司
从 兖州煤业股份有限公司 到 招商证券股份有限公司 的路径：兖州煤业股份有限公司 -> 兖矿东华重工有限公司 -> 兖矿集团唐村实业有限公司 -> 上海兖矿投资有限公司 -> 光大证券股份有限公司 -> 北京同仁堂股份有限公司 -> 中投信用担保有限公司 -> 海通证券股份有限公司 -> 上海证券交易所 -> 中国证券金融股份有限公司 -> 招商证券股份有限公司


In [77]:
dot = draw_circular_path(path_A_to_C, path_C_to_A)
dot.render('circular_holdings_path_2', format='png', cleanup=True)

'circular_holdings_path_2.png'